<a href="https://colab.research.google.com/github/voodoohop/pollinations/blob/dev/colabs/pollinator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Code (Frontend and IPFS Connection)

In [ ]:
#@title Config and Requirements
!pip install coolname gdown papermill json5
from coolname import generate_slug
#ipfs_nodeid = generate_slug(2)#asyncio.run(loop.create_task(get_node_id()))
IPFS_PEER = "ipfs.pollinations.ai"
ipfs_root = "/content/ipfs"
worker_root = "/content"
print(f"---Settings---\nIPFS_ROOT: {ipfs_root}\nIPFS_PEER: {IPFS_PEER}")

In [ ]:
#@title Empty and create ipfs root: */content/ipfs*



!rm -r $ipfs_root
!mkdir -p $ipfs_root/input
!mkdir -p $ipfs_root/output
!mkdir -p $worker_root
!echo "Created IPFS resource" > $ipfs_root/output/log



In [ ]:
#@title Install IPFS

%cd /content
!git clone https://github.com/voodoohop/pollinations.git
%cd /content/pollinations/app
!git pull
!git checkout master 
!git pull
!npm run install_backend
%cd -



In [ ]:
!/content/pollinations/ipfs/ipfsd_install_colab.sh
!nohup  /content/pollinations/ipfs/ipfsd_forever.sh > $ipfs_root/output/log &
!sleep 5



In [ ]:
from os.path import join


notebook_path = join(ipfs_root,"notebook.ipynb")



In [ ]:
node_id=!ipfs id -f="<id>\n"
node_id=node_id[0]

print("---Node ID---:", node_id)

In [ ]:
#@title Create connection to frontend UI

import IPython
from google.colab import output

display(IPython.display.Javascript(f'''
  const iframe = document.createElement("iframe")
  iframe.src = `https://pollinations.ai/incolab.html?node={node_id}`
  document.querySelector("#output-area").appendChild(iframe)
  const iframe2 = document.createElement("iframe")
  iframe2.src = `http://localhost:3000/incolab.html?node={node_id}`
  document.querySelector("#output-area").appendChild(iframe2)
'''))

!sleep 5


In [ ]:
!DEBUG=* pollinate -p {ipfs_root} --ipns --receive --once
!python /content/pollinations/pollinations/enable_papermill_params.py $notebook_path
!nohup pollinate -p {ipfs_root} --ipns --send &

In [ ]:
# !DEBUG=* pollinate -p {ipfs_root} --ipns --send --once

In [ ]:
from glob import glob
from os.path import basename
from json5 import loads


In [ ]:
from subprocess import Popen, PIPE, STDOUT
from sys import stdout
import os

output_path = ipfs_root+"/output"
def run(parameters):
  print("Running",parameters)
 
  parameters["output_path"] = output_path
  _params = [["-p", key, value] for key, value in parameters.items()]
  params = [str(val) for sublist in _params for val in sublist]
  cmd = ["papermill", notebook_path, "/content/notebook_output.ipynb"] + params
  print("Running cmd"," ".join(cmd))
  p = Popen(
          cmd, 
          stdout = PIPE, 
          stderr = STDOUT, 
          shell = False, 
          cwd = worker_root
          )


  while True:
    line = p.stdout.readline()
    if not line: break
    line_text = line.decode("utf-8")

    stdout.write(line)
    with open(f"{ipfs_root}/output/log","a") as log_out:
      if not line_text.startswith("unhandled"):
        log_out.write(line_text)
        log_out.flush()
      
   
        
while True:
  !DEBUG=* pollinate -p {ipfs_root} --ipns --receive --once
  parameters={}
  for path in glob(f"{ipfs_root}/input/*"):
    key = basename(path)
    with open(path,'r') as f: 
      value = f.read()
      parameters[key] = loads(value)
  print("New parameters", parameters)
  run(parameters)
  !sleep 100